In [2]:
import pandas as pd

'''
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
'''

import time
import os
import psycopg2

In [3]:
'''conn = psycopg2.connect(host="localhost",
                        port="5433",
                        database="climate",
                        user="postgres",
                        password="Lululemon=7890")'''

conn = psycopg2.connect(host="sanjose",
                        database="atlas",
                        user="student"
                       )
 
def connect():
    '''
    Connect to the PostgreSQL database server
    '''
    #conn = None
    try:
        # read connection parameters
        #params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        #conn = psycopg2.connect(**params)
      
        # create a cursor
        cur = conn.cursor()
        
   # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 
 
if __name__ == '__main__':
    connect()

OperationalError: could not translate host name "sanjose" to address: Unknown host


In [ ]:
import psycopg2

def create_tables():
    '''
    create tables in the PostgreSQL database
    '''
    commands = (
        """ 
        CREATE TABLE IF NOT EXISTS weather (
                    record_id SERIAL PRIMARY KEY,
                    region TEXT NOT NULL,
                    latitude NUMERIC(6,4) NOT NULL,
                    longitude NUMERIC(6,4) NOT NULL,
                    date DATE,
                    precipitation NUMERIC(5,2),
                    max_temp NUMERIC(4,2),
                    min_temp NUMERIC(4,2),
                    wind NUMERIC(4,2)
                );
        """
        )
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
 
 
if __name__ == '__main__':
    create_tables()

## Part 1: Database Creation

Before running, make sure the file path below points to where all your folders live

In [15]:
directory = 'C:/Users/mohdh/Downloads/Climate Change Database Visual/dir/'  # Dir where all climate change folders live
#db = 'C:/sqlite/psql_schema_test_1.db'  # Target location for created database

### Define Create Table function

In [16]:
'''
arg should ideally be a config() file.
'''
def create_connection():
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    import psycopg2
    conn = None
    try:
        conn = psycopg2.connect(host="localhost",
                                port="5433",
                                database="climate",
                                user="postgres",
                                password="Lululemon=7890")
        return conn
    except:
        logging.warning('unable to connect to database')
        exit(1)
 
    return conn


In [17]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except:
        print('Table creation failed.')


In [33]:
def main():
    
    sql_create_weather_table = """ CREATE TABLE IF NOT EXISTS weather (
                                        record_id SERIAL PRIMARY KEY,
                                        region VARCHAR(64),
                                        latitude NUMERIC,
                                        longitude NUMERIC,
                                        date DATE,
                                        precipitation NUMERIC,
                                        max_temp NUMERIC,
                                        min_temp NUMERIC,
                                        wind NUMERIC
                                    );
                                """
 
    # create a database connection
    conn = create_connection()
 
    # create tables
    if conn is not None:
        # create weather table
        create_table(conn, sql_create_weather_table)
        print('Table created.')
        conn.commit()
        print('Database committed.')
        conn.close()
        print('Database connection closed.')
    else:
        print("Error! cannot create the database connection.")
 
 
if __name__ == '__main__':
    main()

Table created.
Database committed.
Database connection closed.


### Define date extraction functions

In [19]:
def extract_date(year, month, firstday = '01'):
    date = None
    try:
        dlim = '-'
        if len(month) < 2:
            month = '0' + month
        date_list = [year,month,firstday]
        date = (dlim.join(date_list),)
        
        return date
    except:
        print("Enter a valid year / month format!")
        
    return date

def replace_date(line, new_date):
    new_line = None
    try:
        new_line = new_date + tuple([float(l) for l in line[2:]])
        return new_line
    except:
        print("Check that a valid record and date format is passed in.")
    return new_line

### Define INSERT statements

In [20]:
def insert_data(chunk):
    #try:
    # create a database connection
    conn = create_connection()
    cur = conn.cursor()
    with conn:
        args_str = ','.join(cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s)", x).decode("utf-8") for x in tuple(chunk))

        cur.execute("INSERT INTO weather (region,latitude,longitude,date,precipitation,max_temp,min_temp,wind) VALUES " + args_str)
        #cur.executemany(query, chunk)
    conn.commit()
    conn.close()
    #except:
    #    print("Data insertion failed.")

### Define Main functions

In [21]:
def name_split(filename):
    try:
        if len(filename) > 1:
            lat_long = filename.split('_')
            return float(lat_long[1]), float(lat_long[2])
    except:
        print("Check filename is correct!")
        
def combine_tuple(tup1, tup2):
    comb = None
    try:
        comb = tup1 + tup2
        return comb
    except:
        print("Check tuples passed in are correct!")
        
def row_split(cont):
    measurements = None
    try:
        rows = cont.split('\n')
        rows = rows[:-1]
        measurements = [tuple(x.split()) for x in rows]
        return measurements
    except:
        print("Check that file contents are correct!")


In [22]:
def process_file(filename, region, folder_dir):
    data_chunk = None
    try:
        if filename.startswith('data'):

            lat_long = name_split(filename)
            region_co = combine_tuple(region, lat_long)

            with open(folder_dir + '//' + filename, 'r') as f:
                cont = f.read()
                
            measurements = row_split(cont)

            data_chunk = [combine_tuple(region_co, replace_date(x, extract_date(x[0], x[1]))) for x in measurements]
            
            return data_chunk
        else:
            return data_chunk
    except:
        print("Failed to process data for insert.")

In [36]:
def main(directory):
    import time
    start_time = time.time()
    counter = 0
    print('Reading in files from %s' % directory)
    for folder in os.listdir(directory):
        print(str(folder) + " folder in progress.")
        try:
            
            # Assumes that regional folders are formatted in short-form code
            
            if len(folder) < 7:
                folder_dir = str(directory) + str(folder)
                region = (str(folder),)
                
                for filename in os.listdir(folder_dir):
                    insert_data(process_file(filename, region, folder_dir))
                    counter += 1

                    if (counter % 1000) == 0:
                        print("Still working...")           
                        continue
                    else:
                        continue     
            else:
                continue
        except:
            print("You broke the Internet.")
            
        print(str(folder) + " folder done.")
        
    end_time = time.time()
    total_time = end_time - start_time
        
    print("Congratulations, Mr. Stark. All data successfully extracted from all folders.")
    print("Time elapsed: %.2f minutes" % (total_time/60))


### Test

In [23]:
test_file = directory + 'ark/data_31.1875_-92.6875'

In [27]:
def testing(directory):
    import time
    start_time = time.time()
    counter = 0
    region = (directory.split('/')[-2],)
    filename = directory.split('/')[-1]
    folder_dir = directory.rsplit('/',1)[0]
    
    insert_data(process_file(filename,region,folder_dir))
        
    end_time = time.time()
    total_time = end_time - start_time

    print("Congratulations, Mr. Stark. All data successfully extracted from all folders.")
    print("Time elapsed: %.2f minutes" % (total_time/60))

### Run Main function

In [37]:
if __name__ == '__main__':
    main(directory)

Reading in files from C:/Users/mohdh/Downloads/Climate Change Database Visual/dir/
ark folder in progress.
Still working...
Still working...
Still working...
ark folder done.
cali folder in progress.
Still working...
Still working...
Still working...
cali folder done.
color folder in progress.
Still working...
Still working...
Still working...
Still working...
Still working...
color folder done.
crb folder in progress.
Still working...
Still working...
Still working...
Still working...
Still working...
crb folder done.
east folder in progress.
Still working...
Still working...
Still working...
Still working...
Still working...
Still working...
Still working...
Still working...
east folder done.
glakes folder in progress.
Still working...
Still working...
Still working...
Still working...
glakes folder done.
grb folder in progress.
Still working...
Still working...
grb folder done.
gulf folder in progress.
Still working...
Still working...
Still working...
gulf folder done.
low folder i